In [ ]:
from active_critic.utils.gym_utils import sample_expert_transitions
import torch as th
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gym
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.policies import BaseModel
from typing import Any, Dict, List, Optional, Tuple, Type, TypeVar, Union
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
from active_critic.utils.gym_utils import DummyExtractor
import numpy as np
from active_critic.utils.gym_utils import make_policy_dict
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from gym.wrappers import TimeLimit


In [ ]:
seq_len = 100
env_id = 'push'
policy_dict = make_policy_dict()
max_episode_steps = seq_len
env = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_id][1]]()
env._freeze_rand_vec = False
env = TimeLimit(env=env, max_episode_steps=max_episode_steps)


In [ ]:
import gym
import numpy as np

from stable_baselines3 import SAC

#env = gym.make("Hopper-v3")

model_push = SAC("MlpPolicy", env, verbose=1, extra)
model_push.learn(total_timesteps=1e6, log_interval=4)
model_push.save("sac_push")

model = SAC.load("sac_push")

'''while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()'''

In [21]:
env.reset()

array([ 0.00615235,  0.6001898 ,  0.19430117,  1.        , -0.05696029,
        0.68606236,  0.01987216,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.00615235,  0.6001898 ,
        0.19430117,  1.        , -0.09899571,  0.608002  ,  0.01987216,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.08180277,  0.87259032,  0.01987216])

In [ ]:
env = gym.make("Hopper-v2")


In [22]:
obs = env.reset()
rews = []
while True:
    action, _states = model_push.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    rews.append(reward)
    #env.render()
    if done:
      break

In [23]:
rews_np = np.array(rews)

In [26]:
rews_np[-1]

0.04524008908942322

In [ ]:
(rews_np[1:] - rews_np[:-1]).max()

In [ ]:
def make_env(env):
    def _init():
        env._freeze_rand_vec = False
        #rce = ResetCounterWrapper(env)
        riw = RolloutInfoWrapper(env)
        return riw
    return _init

In [ ]:
def make_vec_env_gym(env, num_cpu, seq_len):
    env = SubprocVecEnv([make_env(env) for i in range(num_cpu)])
    return env

In [ ]:
vec_env = make_vec_env_gym(gym.make('Hopper-v3'), 3, 3)

In [ ]:
obsv = vec_env.reset()

In [ ]:
class DummyGymPolicy(BaseModel):
    def __init__(self, observation_space: gym.spaces.Space, action_space: gym.spaces.Space, features_extractor_class: Type[BaseFeaturesExtractor] = ..., features_extractor_kwargs: Optional[Dict[str, Any]] = None, features_extractor: Optional[nn.Module] = None, normalize_images: bool = True, optimizer_class: Type[th.optim.Optimizer] = th.optim.Adam, optimizer_kwargs: Optional[Dict[str, Any]] = None):
        super().__init__(observation_space, action_space, features_extractor_class, features_extractor_kwargs, features_extractor, normalize_images, optimizer_class, optimizer_kwargs)

    def predict(
        self,
        observation: Union[th.Tensor, Dict[str, th.Tensor]],
        state: Optional[Tuple[np.ndarray, ...]] = None,
        episode_start: Optional[np.ndarray] = None,
        deterministic: bool = False,
    ) -> th.Tensor:

        result = np.array([[0,0,0]]*observation.shape[0])
        result[episode_start] = 1
        return result


In [ ]:
epsisode_start = np.array([True, True, False])
dgp = DummyGymPolicy(vec_env.observation_space, vec_env.action_space, features_extractor=DummyExtractor())

In [ ]:
actions = dgp.predict(observation=obsv, episode_start=epsisode_start)

In [ ]:
transitions = sample_expert_transitions(policy=dgp.predict, env=vec_env, episodes=10)

In [ ]:
tf = 0
for i, transition in enumerate(transitions):
    if transition['dones']:
        tf += 1

In [ ]:
tf

In [ ]:
ttf = 0
for act in transitions:
    if act['acts'][0] == 1:
        ttf+=1
ttf

In [ ]:
for k in transitions[21]['infos']['rollout']:
    print(k)

In [ ]:
obsv.shape

In [ ]:
from active_critic.model_src.transformer import *
import torch as th

In [ ]:
seq_len = 6
batch_size = 2
dim = 3

tms = ModelSetup()
tms.d_hid = 12
tms.d_model = 12
tms.d_output = 2
tms.device = 'cuda'
tms.nhead = 1
tms.nlayers = 2
tms.seq_len = seq_len

tm = TransformerModel(model_setup=tms)

In [ ]:
inpt = th.ones([batch_size, seq_len, dim], device='cuda')

In [ ]:
result = tm.forward(inpt, offset=0)

In [ ]:
result.shape